<div class="c_inline" style="box-shadow: rgba(1, 43, 69, 0) 0px 15px 80px;">
<aside class="inline_logo inline_col">
<a class="logo_anchor" href="#top" data-scroll-to="" title="Scroll to top of page" target="_self" data-cursor="1">
<svg class="anchor_monogram" xmlns="http://www.w3.org/2000/svg" width="141" height="45" fill="none" viewBox="0 0 141 45"><path fill="url(#headerLogoGradient)" fill-rule="evenodd" d="M128.18.796v29.66l12.25-1.44v-5.26l-5.989.702V.063l-6.261.733Zm-14.352 31.339V2.477l6.262-.735V26.14l5.988-.703v5.258l-12.25 1.44Zm-26.48 3.101 4.976-.584 4.824-30.215-5.64.662-2.176 15.492-1.867-15.025-4.121.483-1.867 15.457L79.57 6.48l-6.144.72 4.781 29.11 5.25-.62 1.945-13.499 1.946 13.045ZM48.46 39.788v-29.66l6.223-.729v29.66l-6.223.73Zm-40.2 5.075C3.517 45.427.25 42.985.25 39.98v-3.966l5.832-.684v2.781c0 1.113.7 1.845 1.906 1.704.512-.06.903-.226 1.18-.503.215-.213.363-.492.45-.837l.027-.143.027-.162.023-.27.004-.212c0-2.443-1.672-4.253-4.394-6.494C2.504 28.853.602 25.962.602 22.587c0-4.04 2.8-7.555 7.816-8.143 1.848-.217 3.508.155 4.777.902 1.563.918 2.532 2.404 2.532 4.061v3.969l-5.872.688v-3.005c0-1.04-.62-1.634-1.437-1.539-.816.096-1.637.859-1.637 1.897 0 2.226.664 2.889 3.774 6.01l.375.376c2.398 2.386 4.953 4.924 4.953 8.937l-.012.455-.035.425-.059.396-.043.248a7.995 7.995 0 0 1-2.03 3.927 8.521 8.521 0 0 1-1.786 1.444 8.829 8.829 0 0 1-3.656 1.228ZM41.966 10.89l-7.426.869-4.125 30.141 5.516-.646.468-3.784-2.566-.595 5.488-.643.395 4.588 6.453-.75-4.203-29.18Zm-4.942 20.338 1.168-9.887.79 9.659-1.958.228Zm62.262-27.05 12.25-1.432v4.893l-9.101 1.066 3.074.72v5.668l3.965-.464v5.264l-3.965.463v7.49l6.027-.705v5.263l-12.25 1.436V4.178ZM69.613 22.964c1.594-.965 2.606-2.157 2.606-3.569V12.76c0-3.78-3.617-4.989-6.613-4.639l-7.622.892v29.66l6.223-.729V25.97l.816-.096c.856-.101 1.286.376 1.286.999v6.229l.011 4.595 6.012-.705v-11.08c0-.726-.297-1.329-.805-1.823-.476-.467-1.14-.838-1.914-1.126Zm-3.46-3.3c0 .852-.395 1.23-1.051 1.314l-.895.104v-6.627l-2.574-.603 3.469-.406c.66-.077 1.05.21 1.05 1.063v5.154Zm-49.34-.75v-5.079l14.503-1.698v5.078l-6.617.774 2.574.6v23.68l-6.222.728v-24.58l-4.238.496Z" clip-rule="evenodd"></path><defs><linearGradient id="headerLogoGradient" x1="3.429" x2="138.802" y1="30.407" y2="11.543" gradientUnits="userSpaceOnUse"><stop stop-color="#EB512A"></stop><stop offset=".25" stop-color="#EB5429"></stop><stop offset=".44" stop-color="#EC5E24"></stop><stop offset=".62" stop-color="#ED6F1D"></stop><stop offset=".79" stop-color="#EE8713"></stop><stop offset=".95" stop-color="#F0A505"></stop><stop offset="1" stop-color="#F1B100"></stop></linearGradient></defs></svg> </a>
</aside>
</div>
<b>Analyst Notebook & API Reference</b><br><br>
Elevate your SOC, incident response, and threat hunting teams to legendary defenders with Stairwell’s AI-driven cybersecurity automation platform. This Jupyter Notebook was built to introduce some of the capabilities available using the <a href="https://api.stairwell.com">Stairwell APIs</a>.<br>

In [196]:
# If you wish to keep your credentials in a separate file, simply put your ENV_ID and API_KEY
# into a python file and import it like we did here in the same directory. The file is named swcreds.py
# Otherwise, input them manually below
# import swcreds
import sys
import re
import hashlib
import requests
import urllib.parse
import pandas as pd
import json
import pprint
from pandas import json_normalize
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None)

ENV_ID = None      # Put your environment_id here in quotes
API_KEY = None     # Put your API token here in quotes
BASE_URI = "https://app.stairwell.com/v1"
DEFAULT_ASSET_ID = None
UPLOAD_URL = "https://http.intake.app.stairwell.com/v2021.05/upload"
headers = {
    "Content-Type": "application/json; charset=utf-8", 
    "x-api-source": "Stairwell Python Jupyter Notebook",
    'Authorization': API_KEY}


### Helper functions
We have created some simple helper functions that will keep the code below neat and easy to read. Just run the cell below to load them up as you will use them throughout the notebook. Feel free to come back here later and pry into them if you are interested.<br><br>
For an entire breakout of our API endpoints visit __[https://api.stairwell.com](https://api.stairwell.com)__ OR jump to the [bottom of this notebook](#Objects) to try each one individually.

In [251]:
def json_display(dict):
    return json.dumps(json.loads(dict), indent=4)

def default_asset():
    response = requests.get(BASE_URI + '/environments/' + ENV_ID + '/assets', headers={ 'Authorization': API_KEY})
    results = json.loads(response.content)
    for asset in results['assets']:
        if asset['label'] == '__DefaultAsset__':
            default_asset = re.findall("(?:[a-zA-Z0-9\-]{29})", asset['name'])
    return default_asset[0]
DEFAULT_ASSET_ID = default_asset()

def upload_to_stairwell(file, detonate=False):
    try:
        with open(file, "rb") as f:
            file_bytes = f.read()
            sha256 = hashlib.sha256(file_bytes).hexdigest()
            byte_size = sys.getsizeof(file_bytes)
            # Trim the file path so its pretty in the UI
            trimmed_name = file.partition("inception/")[-1]
        stage_1_payload = make_stage1_payload(
            DEFAULT_ASSET_ID, trimmed_name, sha256
        )
        stage_1_response = stage_1_request(stage_1_payload)
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(stage_1_response)
        stage_1_action = stage_1_response["fileActions"][0]["action"]
        if stage_1_action == "UPLOAD":
            stage_2_response = stage_2_request(file_bytes, stage_1_response, detonate)
            print(file)
            print(trimmed_name)
            print(stage_2_response)
        else:
            print(file)
            print(trimmed_name)
            print(stage_1_action)
    except:
        print(f"Failed to import: {file}")
    return

def make_stage1_payload(asset_id, file_name, sha256, md5):
    """Creates a stage 1 payload to send to the intake API"""
    return {
        "asset": {
            "id": asset_id,
        },
        "files": [
            {
                "filePath": file_name,
                "expected_attributes": {
                    "identifiers": [{"sha256": sha256}]
                },
            }
        ],
    }

def stage_1_request(stage_1_payload):
    response = requests.request(
        "POST",
        UPLOAD_URL,
        data=json.dumps(stage_1_payload),
        timeout=300,
    )
    return response.json()

def stage_2_request(file_bytes, stage_1_response, detonate):
    stage_2_payload = stage_1_response["fileActions"][0]["fields"]
    if detonate == True:
        stage_2_payload['x-goog-meta-file-detonate'] = 'DETONATE'
    stage_2_payload["file"] = file_bytes
    # Stage 2 attempt upload
    upload_url = stage_1_response["fileActions"][0]["uploadUrl"]
    print(upload_url)
    response_2 = requests.request(
        "POST", upload_url, files=stage_2_payload, timeout=300
    )
    return response_2

def extract_rules(text):
    parsed_rules = []
    if text != None:
        for s in text:
            parsed_rules.append(s.split('/yaraRules/')[-1].strip())
    return parsed_rules

def extract_verdict(opinions_list):
    if isinstance(opinions_list, list) and len(opinions_list) > 0 and opinions_list != None:
        return opinions_list[0].get('verdict')
    else:
        return 'No opinion'

def extract_tags(tags):
    if isinstance(tags, list) and len(tags) > 0 and tags != None:
        tags_list = []
        for t in tags:
            tags_list.append(t['value'])
        return tags_list
    
def get_all_assets():
    response = requests.get(BASE_URI + '/environments/' + ENV_ID + '/assets', headers={ 'Authorization': API_KEY})
    results = json.loads(response.content)
    return results

def detonate_file(hash):
    response = requests.post(BASE_URI + '/objects/' + hash + '/detonation:trigger', headers={ 'Authorization': API_KEY}, data=json.dumps({'parent':hash}))
    results = json.loads(response.content)
    print("If the results below are empty please allow a few minutes for detonation to complete.")
    return results

def get_detonation(hash):
    response = requests.get(BASE_URI + '/objects/' + hash + '/detonation', headers={ 'Authorization': API_KEY})
    results = json.loads(response.content)
    return results

def get_object_metadata(hash):
    meta_response = requests.get(BASE_URI + '/objects/' + hash + '/metadata', headers={ 'Authorization': API_KEY})
    check_response = meta_response.text.find("no object found")
    if check_response != -1:
        print(f"Object {hash} did not have any matching results.")
    else:
        results = json.loads(meta_response.content)
        return results
    
def get_object_opinions(hash):
    op_response = requests.get(BASE_URI + '/objects/' + hash + '/opinions', headers={ 'Authorization': API_KEY})
    op_results = json.loads(op_response.content)
    return op_results

def get_object_sightings(hash):
    objs_response = requests.get(BASE_URI + '/objects/' + hash + '/sightings', headers={ 'Authorization': API_KEY})
    objs_results = json.loads(objs_response.content)
    return objs_results

def get_variant_sightings(list):
    seen_list = []
    for s in list:
        vars_response = requests.get(BASE_URI + '/objects/' + hash + '/sightings', headers={ 'Authorization': API_KEY})
        vars_results = json.loads(vars_response.content)
        if len(vars_results['objectSightings']) > 0:
            seen_list.append(s)
    return seen_list

def get_object_tags(hash):
    tags_response = requests.get(BASE_URI + '/objects/' + hash + '/tags', headers={ 'Authorization': API_KEY})
    tags_results = json.loads(tags_response.content)
    return tags_results

def get_asset_label(asset_id):
    assetMeta_response = requests.get(BASE_URI + '/assets/' + asset_id, headers=headers)
    if json.loads(assetMeta_response.content)['label'] != None:
        return json.loads(assetMeta_response.content)['label']
    else:
        return ""

def get_object_comments(hash):
    comms_response = requests.get(BASE_URI + '/objects/' + hash + '/comments', headers={ 'Authorization': API_KEY})
    comms_results = json.loads(comms_response.content)
    return comms_results

def get_object_variants_list(hash):
    variants_list = []
    variants_response = requests.get(BASE_URI + '/objects/' + hash + '/variants', headers=headers)
    variants_results = json.loads(variants_response.content)
    if len(variants_results['objectVariants']) > 0:
        for v in variants_results['objectVariants']:
            variants_list.append(v['variant']['sha256'])
        return variants_list
    else:
        return None

def ioc_variant_analysis(vsightings_list):
    dfv = pd.DataFrame()
    for vs in vsightings_list:
        metadata = get_object_metadata(vs)
        opinions = get_object_opinions(vs)
        sightings = get_object_sightings(vs)
        comments = get_object_comments(vs)
        dfm = json_normalize(metadata)
        dfo = json_normalize(opinions)
        dfs = json_normalize(sightings)
        dfc = json_normalize(comments)
        dfm['opinions'] = dfo['opinions']
        dfm['sightings'] = dfs['objectSightings']
        dfm['comments'] = dfc['comments']
        if len(sightings['objectSightings']) > 0:
            dfm['file'] = sightings['objectSightings'][0]['filepath'] + sightings['objectSightings'][0]['filename']
        else:
            dfm['file'] = None
        dfv = pd.concat([dfv, dfm])

    dfv['yaraRuleMatches'] = dfv['yaraRuleMatches'].apply(lambda x: extract_rules(x))
    dfv['tags'] = dfv['tags'].apply(lambda v: extract_tags(v))
    dfv['opinions'] = dfv['opinions'].apply(extract_verdict)
    dfv = dfv.drop(columns=['name', 'sha3256', 'shannonEntropy', 'imphash', 'imphashSorted', 'environments','networkIndicators.improbableHostnames','networkIndicators.privateHostnames','networkIndicators.uninterestingIpAddresses','objectSignature.pkcs7VerificationResult','objectSignature.x509Certificates'])
    dfv = dfv.reindex(['file', 'sha256','opinions','yaraRuleMatches','malEval.labels','malEval.probabilityBucket','tags','malEval.severity','comments', 'sightings', 'size','magic','mimeType','stairwellFirstSeenTime','detonation.tags','detonation.overview','detonation.sampleId','networkIndicators.ipAddresses','networkIndicators.hostnames'], axis=1)
    return dfv

Here we create a list of some hashes we want to analyze.

In [263]:
# Input Hashes
iocs = [
    "96b77284744f8761c4f2558388e0aee2140618b484ff53fa8b222b340d2a9c84",
    "05563f41e711850f3ea8747a359a1f9d3127bb26a59dc69af4ae0eb3a04e5c51",
    "e7762f90024c5366807c7c145d3456f0ac3be086c0ec3557427d3c2c10a2052d",
    "1d3b5c650533d13c81e325972a912e3ff8776e36e18bca966dae50735f8ab296"
]

We check the hashes against our GetObjectMetadata, ListOpinions, ListComments, and ListObjectSightings endpoints as a baseline and look for any variants in our environment. We created some helper functions to do all this and make the code easier to read. As you can see, there is a lot going on here. Note many of the data columns were dropped for the sake of screen space. To see the entire response from each of these endpoints please see the section at the bottom of this notebook.

In [267]:
affected_assets = []
def ioc_object_analysis(ioc_list):
    global affected_assets
    df = pd.DataFrame()
    dfv = pd.DataFrame()
    for i in ioc_list:
        variants = get_object_variants_list(i)
        seen_variants = get_variant_sightings(variants)
        if len(seen_variants)> 0:
            dfv = ioc_variant_analysis(seen_variants)
        
        metadata = get_object_metadata(i)
        opinions = get_object_opinions(i)
        sightings = get_object_sightings(i)
        comments = get_object_comments(i)
        dfm = json_normalize(metadata)
        dfo = json_normalize(opinions)
        dfs = json_normalize(sightings)
        dfc = json_normalize(comments)
        dfm['opinions'] = dfo['opinions']
        dfm['sightings'] = dfs['objectSightings']
        dfm['comments'] = dfc['comments']
        if len(sightings['objectSightings']) > 0:
            dfm['file'] = sightings['objectSightings'][0]['filepath'] + sightings['objectSightings'][0]['filename']
            for a in sightings['objectSightings']:
                affected_assets.append(get_asset_label(a['asset'].split("/")[1]))
        else:
            dfm['file'] = None
        if dfv.empty:
            df = pd.concat([df, dfm])
        else:
            df = pd.concat([df, dfm, dfv])

    df['yaraRuleMatches'] = df['yaraRuleMatches'].apply(lambda x: extract_rules(x))
    df['tags'] = df['tags'].apply(lambda v: extract_tags(v))
    df['opinions'] = df['opinions'].apply(extract_verdict)
    df = df.drop(columns=['name', 'sha3256', 'shannonEntropy', 'imphash', 'imphashSorted', 'environments','networkIndicators.improbableHostnames','networkIndicators.privateHostnames','networkIndicators.uninterestingIpAddresses','objectSignature.pkcs7VerificationResult','objectSignature.x509Certificates'])
    df = df.reindex(['file', 'sha256','opinions','yaraRuleMatches','malEval.labels','malEval.probabilityBucket','tags','malEval.severity','comments', 'sightings', 'size','magic','mimeType','stairwellFirstSeenTime','detonation.tags','detonation.overview','detonation.sampleId','networkIndicators.ipAddresses','networkIndicators.hostnames'], axis=1)
    return df

ioc_object_analysis(iocs) # Takes an iterable (list, tuple, set, etc.)

,file,sha256,opinions,yaraRuleMatches,malEval.labels,malEval.probabilityBucket,tags,malEval.severity,comments,sightings,size,magic,mimeType,stairwellFirstSeenTime,detonation.tags,detonation.overview,detonation.sampleId,networkIndicators.ipAddresses,networkIndicators.hostnames
0,None,96b77284744f8761c4f2558388e0aee2140618b484ff53fa8b222b340d2a9c84,No opinion,[],[trojan.hermeticwiper],PROBABILITY_VERY_HIGH,None,INFORMATIONAL,[],[],17480,"PE32+ executable (native) x86-64, for MS Windows",application/x-dosexec,2022-02-26T04:15:26.193531Z,[],,,[],"[crl.microsoft.com, crl.verisign.com, csc3-2010-aia.verisign.com, csc3-2010-crl.verisign.com, logo.verisign.com, ocsp.verisign.com, www.verisign.com]"
0,C:\prln.exe,05563f41e711850f3ea8747a359a1f9d3127bb26a59dc69af4ae0eb3a04e5c51,No opinion,"[Methodology_APIHashing_rol7XorHash32, Methodology_StackStrings_Misc_2, Methodology_Mutation_URL_XOR_PE, Methodology_FilePackage_Nesting_PE_In_PE_, SUSP_XORed_URL_in_EXE, TTP_FilePackage_PE_embedded_unsigned]","[encoder, Gen:Variant.Ransom.UniWinniCrypt.3, malware-ml]",PROBABILITY_VERY_HIGH,None,HIGH,[],"[{'sightingTime': '2023-08-28T20:04:56.681472158Z', 'environment': 'BJJKNH-MCGB8F-WAED6B-3ELG8CPJ', 'asset': 'assets/JHUQCW-JU56FW-ZC9WC8-N2655HLN', 'filename': 'prln.exe', 'filepath': 'C:\'}]",1144320,"PE32 executable (GUI) Intel 80386, for MS Windows",application/x-dosexec,2023-02-15T20:30:15.434022Z,[],,,[],[]
0,c:\windows\system32\wlnida02s.dll,e7762f90024c5366807c7c145d3456f0ac3be086c0ec3557427d3c2c10a2052d,MALICIOUS,"[wiper_HermeticWiper_variants, MAL_HERMETIC_WIPER, HermeticWiper, Pro_apt_HermeticWiper, Pro_apt_HermeticWiper_certificate, HermeticWiper_Wide_DriveStuff_3, HermeticWiper_Wide_DriveStuff_2, HermeticWiper_Wide_DriveStuff, TTP_Signer_VersionInfo_Mismatch, TTP_Signer_VersionInfo_Mismatch_Pt_Deux, TTP_Signer_VersionInfo_Mismatch_NoStr]","[trojan.hermeticwiper, Generic.HermeticWiper.A.EFA52CE3, generic.hermeticwiper, hermeticwiper, malware]",PROBABILITY_VERY_HIGH,None,HIGH,[],"[{'sightingTime': '2022-05-29T13:07:12.464263Z', 'environment': 'BJJKNH-MCGB8F-WAED6B-3ELG8CPJ', 'asset': 'assets/KH9323-73UXL2-X22G9M-A923P3LW', 'filename': 'wlnida02s.dll', 'filepath': 'c:\windows\system32\'}]",118024,"PE32 executable (GUI) Intel 80386, for MS Windows",application/x-dosexec,2022-05-29T02:07:12.464263Z,[],,,[],"[cacerts.digicert.com, crl3.digicert.com, crl4.digicert.com, ocsp.digicert.com, qxq1t.lc, www.digicert.com]"
0,/home/devadmin/Documents/wrt,1d3b5c650533d13c81e325972a912e3ff8776e36e18bca966dae50735f8ab296,MALICIOUS,"[Keyhunt__RSA_PRIVATE_KEY, Akira_strings, Hunt_File_Discovery, Hunt_TimeStomp]","[suspicious, Trojan.Linux.Ransom.298176]",PROBABILITY_VERY_HIGH,"[akira, ransomware]",HIGH,"[{'body': 'Akira ', 'email': 'foster@stairwelldemo.com', 'createTime': '2023-08-07T17:57:16.716Z', 'environment': 'BJJKNH-MCGB8F-WAED6B-3ELG8CPJ'}]","[{'sightingTime': '2023-08-10T02:09:57.395646061Z', 'environment': 'BJJKNH-MCGB8F-WAED6B-3ELG8CPJ', 'asset': 'assets/YV253W-WFVL8L-XVRAQ3-RXRA9ZNN', 'filename': 'wrt', 'filepath': '/home/devadmin/Documents/'}]",3970960,"ELF 64-bit LSB executable, x86-64, version 1 (GNU/Linux)",application/x-executable,2023-06-24T01:51:33.133545Z,[],,,[],"[bugs.launchpad.net, dkuug.dk, gcc.gnu.org, www.torproject.org]"


The cell above recusively combines data from several endpoints to tell you lots of relevant information such as the probability a given file is malicious, severity, tags, comments, file path (if available), variants, sightings, and much more. Once a file is found that has a malicious score, the next question usually leads to something like "Has it ever been seen in my environment?"<br><br> Run the next cell to see if any assets (endpoints) in your environment have ever observed any of the file hashes above *as well as* any variant of those files.

In [266]:
print(affected_assets)

['JUNGLE', 'olivia-win11', 'dev-build']


You've just performed a mini breach analysis in mere seconds! Using code/workflow like this, automation takes on a whole new level of usefulness by performing a tremendous amount of triage and analysis that can be packaged up and sent into a SIEM, ticketing platform, or case management system for further actioning.
___

If you would like to remotely detonate a file already in the platform, run the cell below.

In [ ]:
hash = 'df28459618ca4dc29ae2f60be835dbe2046029e7732e42a1669ac113d847b659'
print(json.dumps(detonate_file(hash), indent=4))

To see the results of the detonation, wait a couple minutes and then get the report (WARNING, this will potentially be a large amount of data)

In [44]:
hash = 'df28459618ca4dc29ae2f60be835dbe2046029e7732e42a1669ac113d847b659'
print(json.dumps(get_detonation(hash), indent=4))

If you want to upload a sample for analysis, make sure the file is in the same directory as this notebook and enter it's information below, then run the cell.<br>
TIP: If you would like to detonate the file as soon as it's uploaded, set `detonate=True`

In [ ]:
upload_to_stairwell("file.exe", detonate=False)

<div class="alert alert-block alert-info">
<b>Individual API Examples:</b> Below you will find examples of our individual endpoints so you can explore the full data returned from each one.
</div>

## [Objects](#Objects)

### GetObjectMetadata
Gets the metadata for the object specified (sha256).

In [ ]:
objMeta_response = requests.get(BASE_URI + '/objects/' + hash + '/metadata', headers=headers)
print(json_display(objMeta_response.content))

### ListObjectMetadata
Fetches a list of object metadata. Objects returned match the CEL filter query specified in the request.

__[https://help.stairwell.com/en/knowledge/how-do-i-write-a-cel-query](https://help.stairwell.com/en/knowledge/how-do-i-write-a-cel-query)__

In [ ]:
# Copy the query from the UI and paste it in the urllib.parse.quote_plus() function to URL encode it
query = urllib.parse.quote_plus('mal_eval.malicious == true && asset.count > 0 && asset.count < 10 && object.magic == "EXE"')
objMetaList_response = requests.get(BASE_URI + '/objects/metadata?filter=' + query, headers=headers)
print(json_display(objMetaList_response.content))

### ListObjectVariants
Gets the variants for the specified object.

In [ ]:
variants_response = requests.get(BASE_URI + '/objects/' + hash + '/variants', headers=headers)
print(json_display(variants_response.content))

### ListObjectSightings
Gets the sightings for the specified object an any assets on which it has appeared in your environment.

In [ ]:
sightings_response = requests.get(BASE_URI + '/objects/' + hash + '/sightings', headers=headers)
print(json_display(sightings_response.content))

### ListObjectOpinions
Gets the opinions for the specified object.

In [ ]:
opinions_response = requests.get(BASE_URI + '/objects/' + hash + '/opinions', headers=headers)
print(json_display(opinions_response.content))

### CreateObjectOpinion
Creates/Updates a new opinion for a given object. Permitted 'verdicts' include:<br>
`"OPINION_VERDICT_UNSPECIFIED"`<br>
`"NO_OPINION"`<br>
`"TRUSTED"`<br>
`"GRAYWARE"`<br>
`"MALICIOUS"`<br>

In [ ]:
body = {"verdict": "OPINION_VERDICT_UNSPECIFIED", "environment": ENV_ID}
opinions_response = requests.request("POST", BASE_URI + '/objects/' + hash + '/opinions', headers=headers, data=json.dumps(body))
print(json_display(opinions_response.content))

### ListObjectComments
Gets the comments for the specified object.

In [ ]:
comms_response = requests.get(BASE_URI + '/objects/' + hash + '/comments', headers=headers)
print(json_display(comms_response.content))

### CreateObjectComment
Appends a new comment to a given object

In [ ]:
body = {"body": "This is a test comment", "environment": ENV_ID}
comms_response = requests.request("POST", BASE_URI + '/objects/' + hash + '/comments', headers=headers, data=json.dumps(body))
print(json_display(comms_response.content))

### ListObjectTags
Gets the tags for the specified object.

In [ ]:
tags_response = requests.get(BASE_URI + '/objects/' + hash + '/tags', headers=headers)
print(json_display(tags_response.content))

### DeleteObjectTag
Deletes a tag on a given object. Returns a blank response & RC 200 if successful


In [ ]:
tag_id = None
tags_response = requests.request("DELETE", BASE_URI + '/objects/' + hash + '/tags/' + tag_id, headers=headers)
print(json_display(tags_response.content))

### CreateObjectTag
Adds a new tag to a given object

In [ ]:
body = {"value": "TEST_TAG", "environment": ENV_ID}
tags_response = requests.request("POST", BASE_URI + '/objects/' + hash + '/tags', headers=headers, data=json.dumps(body))
tag_json = json.loads(tags_response.content)
tag_id = tag_json['name'].split("/")
print(f" the tag_id is: {tag_id[3]}")
print(json_display(tag_json))

### TriggerObjectDetonation
Triggers a new detonation for the parent object.

In [ ]:
body = {"parent": hash}
detonation_response = requests.request("POST", BASE_URI + '/objects/' + hash + '/detonation:trigger', headers=headers, data=json.dumps(body))
print(json_display(detonation_response.content))

### GetObjectDetonation
Gets the detonation data for a given object if a detonation has already successfully completed.

In [ ]:
detonation_response = requests.get(BASE_URI + '/objects/' + hash + '/detonation', headers=headers)
print(json_display(detonation_response.content))

### DownloadObject
Downloads the object as file.

In [ ]:
download_response = requests.get(BASE_URI + '/objects/' + hash + ':download', headers=headers)
file = download_response.content

___
## [Hostnames](#Hostnames)

### GetHostnameMetadata
Gets metadata for a particular hostname.

In [ ]:
hostname = "stairwell.com"
hostMeta_response = requests.get(BASE_URI + '/hostnames/' + hostname + '/metadata', headers=headers)
print(json_display(hostMeta_response.content))

### ListHostnameOpinions
Gets the opinions for a given hostname.

In [ ]:
hostOpinions_response = requests.get(BASE_URI + '/hostnames/' + hostname + '/opinions', headers=headers)
print(json_display(hostOpinions_response.content))

### CreateHostnameOpinion
Updates/Creates the opinion of a given hostname. Permitted 'verdicts' include:<br>
`"OPINION_VERDICT_UNSPECIFIED"`<br>
`"NO_OPINION"`<br>
`"TRUSTED"`<br>
`"GRAYWARE"`<br>
`"MALICIOUS"`<br>

In [ ]:
body = {"verdict": "OPINION_VERDICT_UNSPECIFIED", "environment": ENV_ID}
hostOpinions_response = requests.request("POST", BASE_URI + '/hostnames/' + hostname + '/opinions', headers=headers, data=json.dumps(body))
print(json_display(hostOpinions_response.content))

### ListHostnameComments
Gets the comments for a given hostname

In [ ]:
hostComms_response = requests.get(BASE_URI + '/hostnames/' + hostname + '/comments', headers=headers)
print(json_display(hostComms_response.content))

### CreateHostnameComment
Appends a new comment to a given hostname.

In [ ]:
body = {"body": "This is a test comment", "environment": ENV_ID}
hostComms_response = requests.request("POST", BASE_URI + '/hostnames/' + hostname + '/comments', headers=headers, data=json.dumps(body))
print(json_display(hostComms_response.content))

### ListHostnameTags
Gets the tags for the specified hostname.

In [ ]:
hostTags_response = requests.get(BASE_URI + '/hostnames/' + hostname + '/tags', headers=headers)
print(json_display(hostTags_response.content))

### CreateHostNameTag
Adds a new tag to a given hostname

In [ ]:
body = {"value": "TEST_TAG2", "environment": ENV_ID}
hostTags_response = requests.request("POST", BASE_URI + '/hostnames/' + hostname + '/tags', headers=headers, data=json.dumps(body))
hostTag_json = json.loads(hostTags_response.content)
hostTag_id = tag_json['name'].split("/")
print(f" the tag_id is: {hostTag_id[3]}")
print(json_display(hostTags_response.content))

### DeleteHostNameTag
Removes a specified tag from a given hostname.

In [ ]:
hostTags_response = requests.request("DELETE", BASE_URI + '/hostnames/' + hostname + '/tags/' + tag_id, headers=headers)
print(json_display(hostTags_response.content))

___
## [IP Addresses](#IpAddresses)

### GetIpAddressMetadata
Gets metadata for a particular IP address.

In [ ]:
ipAddress = "8.8.8.8"
ipMeta_response = requests.get(BASE_URI + '/ipAddresses/' + ipAddress + '/metadata', headers=headers)
print(json_display(ipMeta_response.content))

### ListIpAddressOpinions
Gets the opinions for a given IP address.

In [ ]:
ipOpinions_response = requests.get(BASE_URI + '/ipAddresses/' + ipAddress + '/opinions', headers=headers)
print(json_display(ipOpinions_response.content))

### CreateIpAddressOpinion
Updates/Creates the opinion of a given IP address. Permitted 'verdicts' include:<br>
`"OPINION_VERDICT_UNSPECIFIED"`<br>
`"NO_OPINION"`<br>
`"TRUSTED"`<br>
`"GRAYWARE"`<br>
`"MALICIOUS"`<br>

In [ ]:
body = {"verdict": "OPINION_VERDICT_UNSPECIFIED", "environment": ENV_ID}
ipOpinions_response = requests.request("POST", BASE_URI + '/ipAddresses/' + ipAddress + '/opinions', headers=headers, data=json.dumps(body))
print(json_display(ipOpinions_response.content))

### ListIpAddressComments
Gets the comments for a given IP address

In [ ]:
ipComms_response = requests.get(BASE_URI + '/ipAddresses/' + ipAddress + '/comments', headers=headers)
print(json_display(ipComms_response.content))

### CreateIpAddressComment
Appends a new comment to a given IP address.

In [ ]:
body = {"body": "This is a test comment", "environment": ENV_ID}
ipComms_response = requests.request("POST", BASE_URI + '/ipAddresses/' + ipAddress + '/comments', headers=headers, data=json.dumps(body))
print(json_display(ipComms_response.content))

### ListIpAddressTags
Gets the tags for the specified IP Address.

In [ ]:
ipTags_response = requests.get(BASE_URI + '/ipAddresses/' + ipAddress + '/tags', headers=headers)
print(json_display(ipTags_response.content))

### CreateIpAddressTag
Adds a new tag to a given IP Address

In [ ]:
body = {"value": "TEST_TAG", "environment": ENV_ID}
ipTags_response = requests.request("POST", BASE_URI + '/ipAddresses/' + ipAddress + '/tags', headers=headers, data=json.dumps(body))
ipTag_json = json.loads(ipTags_response.content)
ipTag_id = ipTag_json['name'].split("/")
print(f" the tag_id is: {ipTag_id[3]}")
print(json_display(ipTags_response.content))

### DeleteIpAddressTag
Removes a specified tag from a given IP address.

In [ ]:
tag_id = "A1B2C43D4E5F6===" # Example tag_id
ipTags_response = requests.request("DELETE", BASE_URI + '/ipAddresses/' + ipAddress + '/tags/' + tag_id, headers=headers)
print(json_display(ipTags_response.content))

___
## [Yara Rules](#YaraRules)

<div class="alert alert-block alert-info">
<b>Tip:</b> Note that Yara Rules are specific to the environment they were created in or shared from, if a rule originates from an environment besides your own, you would need to supply that environment_id as well when requesting the rule metadata.<br>
- Florian's Open Rules = NCS2SM-YHB2KT-SAFUDX-JC7F6WYA<br>
- Stairwell OSINT Rules = EB3DXY-3ZYFVH-6HNKJQ-GAPKHESS<br>
- Stairwell Methodology Rules = GEG6FU-MRARGF-TLTM6X-H6MGDT5E<br>
- Stairwell Research Rules = NQNJM6-5LSCAF-3MC5FJ-W8EKGW6N<br><br>
<b>Note that you cannot make changes to rules shared from outside your environment</b>
</div>

### GetYaraRule
Gets Yara rule metadata and rule definition.

In [ ]:
RuleEnv_ID = "EB3DXY-3ZYFVH-6HNKJQ-GAPKHESS"
yaraRule = 'Hunt_File_Discovery'
yaraRule_response = requests.get(BASE_URI + '/environments/' + RuleEnv_ID + '/yaraRules/' + yaraRule, headers=headers)
print(json_display(yaraRule_response.content))

### ListYaraRuleTags
Gets the tags for the specified yara rule.

In [ ]:
yaraRule_response = requests.get(BASE_URI + '/environments/' + RuleEnv_ID + '/yaraRules/' + yaraRule + '/tags', headers=headers)
print(json_display(yaraRule_response.content))

### CreateYaraRuleTag
Create and apply a tag to a given Yara rule.

In [ ]:
body = {"value": "TEST_TAG", "environment": RuleEnv_ID}
yaraRuleTags_response = requests.request("POST", BASE_URI + '/environments/' + RuleEnv_ID + '/yaraRules/' + yaraRule + '/tags', headers=headers, data=json.dumps(body))
yaraRuleTags_json = json.loads(yaraRuleTags_response.content)
yaraRuleTags_id = yaraRuleTags_json['name'].split("/")
print(f" the tag_id is: {yaraRuleTags_id[3]}")
print(json_display(yaraRuleTags_response.content))

### DeleteYaraRuleTag
Deletes/Removes a specific tag for a given Yara rule in your environment. No changes can be made to Yara rules from other shared environments.

In [ ]:
tag_id = "A1B2C43D4E5F6===" # Example tag_id
yaraRuleTags_response = requests.request("DELETE", BASE_URI + '/environments/' + RuleEnv_ID + '/yaraRules/' + yaraRule + '/tags/' + tag_id, headers=headers)
print(json_display(yaraRuleTags_response.content))

___
## [Assets](#Assets)

### GetAssetMetadata
Gets metadata for a particular asset.

In [ ]:
asset_id = "AAAAAA-BBBBBB-CCCCCC-12345678" # Example asset_id (this will produce an error if not given an actual asset_id)
assetMeta_response = requests.get(BASE_URI + '/assets/' + asset_id, headers=headers)
print(json_display(assetMeta_response.content))

### ListEnvironmentAssets
Lists all the assets for a given environment_id

In [ ]:
response = requests.get(BASE_URI + '/environments/' + ENV_ID + '/assets', headers=headers)
print(json_display(response.content))

### ListAssetTags
Gets the tags for the specified asset.

In [ ]:
assetTags_response = requests.get(BASE_URI + '/assets/' + asset_id + '/tags', headers=headers)
print(json_display(assetTags_response.content))

### CreateAssetTag
Creates and applies a new tag to a given asset

In [ ]:
body = {"value": "TEST_TAG", "environment": ENV_ID}
assetTags_response = requests.request("POST", BASE_URI + '/assets/' + asset_id + '/tags', headers=headers, data=json.dumps(body))
assetTag_json = json.loads(assetTags_response.content)
assetTag_id = assetTag_json['name'].split("/")
print(f" the tag_id is: {assetTag_id[3]}")
print(json_display(assetTags_response.content))

### DeleteAssetTag
Removes a tag from a given asset

In [ ]:
tag_id = "A1B2C43D4E5F6===" # Example tag_id
assetTags_response = requests.request("DELETE", BASE_URI + '/assets/' + asset_id + '/tags/' + tag_id, headers=headers)
print(json_display(assetTags_response.content))